# COSC-4117EL: Constraint Satisfaction Problem (n-queen)

N-Queens is a CSP problem consists of placing N queens (N > 3) on an N×N chessboard so that no two queens threaten each other. This means that no two queens can be in the same row, column, or diagonal.

# Backtracking search

The `backtracking` function generate a solution to the N-Queens puzzle using a **backtracking search** approach.

The solution works as follows:

The is_valid function checks whether it's safe to place a queen at a given row and column. It checks for threats from previously placed queens in the same row and both diagonals.  The algorithm works by trying to place one queen in each raw iteratively. The `solve_n_queens` function, in particular, attempts to place queens in a cell. Every recursive call to the `solve_n_queens` function essentially advances to the next row. Because of the iterative column placement and the manner in which queens are stored, two queens can never be in the same column. Hence, there's no need to check for threats from queens in the same column; it's implicitly ensured by the algorithm's design.


# AC-3

The `backtracking_ac3` function combines backtracking with the AC-3 algorithms for pruning search spaces.

Variables: Qk , k= 1,2 … N
Domains: {1, 2 , ... N}

In [ ]:
4## Check if it's safe to place a queen at the specified position.
def is_safe(board, row, col, n):
    # We only need to check for queens on the left side as we're placing queens column by column.

    # Check if there's a queen in the same row on the left side.
    for i in range(col):
        if board[row][i] == 1:
            return False

    # Check if there's a queen in the upper-left diagonal.
    for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False

    # Check if there's a queen in the lower-left diagonal.
    for i, j in zip(range(row, n, 1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False

    return True

# Apply AC-3 algorithm to prune the domain of the current column.
def ac3(board, col, n, domains):
    # If we've reached or surpassed the last column, return the domains.
    if col >= n:
        return domains

    pruned = set()

    # Check each value in the current column's domain.
    for value in domains[col]:
        is_consistent = False

        # For each value, see if placing a queen in the next column is safe.
        for next_value in domains[col + 1]:
            board[value][col] = 1
            if is_safe(board, next_value, col + 1, n):
                is_consistent = True
            board[value][col] = 0
            if is_consistent:
                break

        # If not consistent, add the value to the pruned set.
        if not is_consistent:
            pruned.add(value)

    # Update the domain by removing the pruned values.
    domains[col] = list(set(domains[col]) - pruned)
    return domains

# Use backtracking to solve the N Queens problem.
def solve_n_queens(board, col, n, solutions, search_count, use_ac3=False):
    # If we've placed queens in all columns, add the board to solutions.
    if col >= n:
        solutions.append([row[:] for row in board]) # add the found solution
        return  # exits the current recursive call of solve_n_queens, returning control to the previous recursive call in the stack

    # If AC-3 is used and we're not in the last column, prune the domains.
    if use_ac3 and col < n - 1:
        domains = {i: list(range(n)) for i in range(n)}
        ac3(board, col, n, domains)
        possible_rows = domains[col]
    else:
        possible_rows = range(n)

    # For each possible row, if safe, place the queen and recurse.
    for i in possible_rows:
        search_count[0] += 1 # keep track the number of search attempted
        if is_safe(board, i, col, n):
            board[i][col] = 1 # place a queen
            # recursely search for a solution
            solve_n_queens(board, col + 1, n, solutions, search_count, use_ac3)
            board[i][col] = 0 # undone placement, backtrack

# Print all the solutions in a readable format.
def print_solutions(solutions):
    for sol in solutions:
        for row in sol:
            print(' '.join(['Q' if x else '_' for x in row]))
        print("\n" + "-"*20 + "\n")

# Main execution starts here.
import time

n = int(input("Enter the value of N for N-Queens problem: "))
#method = input("Which method to use? (backtracking/ac3): ").strip().lower()
#use_ac3 = method == "ac3"

board = [[0] * n for _ in range(n)]

solutions = []
search_count = [0]
start_time = time.time()
# default backtracking
solve_n_queens(board, 0, n, solutions, search_count)
end_time = time.time()
#print_solutions(solutions)

# Print the results.
print("BACKTRACKING")
duration = (end_time - start_time)*1000
print(f"Total solutions found: {len(solutions)}")
print(f"Total search attempts: {search_count[0]}")
print(f"Runtime (ms):{duration:.6f}")

solutions = []
search_count = [0]
start_time = time.time()
# using ac3
solve_n_queens(board, 0, n, solutions, search_count, "ac3")
end_time = time.time()
#print_solutions(solutions)

# Print the results.
print("AC3")
duration = (end_time - start_time)*1000
print(f"Total solutions found: {len(solutions)}")
print(f"Total search attempts: {search_count[0]}")
print(f"Runtime (ms):{duration:.6f}")


Enter the value of N for N-Queens problem: 10
BACKTRACKING
Total solutions found: 724
Total search attempts: 348150
Runtime (ms):421.889544
AC3
Total solutions found: 724
Total search attempts: 156294
Runtime (ms):2152.664661
